# 19 Jan - GloVe

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

### 1. Loading Data

In [ ]:
#define the training corpus
#I use the Inaugural Address Corpus from NLTK
import nltk
from nltk.corpus import inaugural
corpus = nltk.corpus.inaugural.sents()

In [ ]:
#the corpus is already tokenized
print(corpus[:3])

In [ ]:
#convert the words in the corpus into lower case
corpus_tokenized = [[]] * len(corpus)
for i in range(len(corpus)):
    corpus_tokenized[i] = [word.lower() for word in corpus[i]]

print(corpus_tokenized[:3])

In [ ]:
#remove stop words with SpaCy
import spacy
nlp = spacy.load("en_core_web_sm")
stopwords = nlp.Defaults.stop_words

for sentence in corpus_tokenized:
    for word in sentence[:]:
        if word in stopwords:
            sentence.remove(word)

print(corpus_tokenized[:3])

In [ ]:
#remove punctutations with String
import string
punctutations = string.punctuation

for sentence in corpus_tokenized:
    for word in sentence[:]:
        if word in punctutations:
            sentence.remove(word)

print(corpus_tokenized[:3])

In [ ]:
#remove '--'
for sentence in corpus_tokenized:
    for word in sentence[:]:
        if word == '--':
            sentence.remove(word)

print(corpus_tokenized[:3])

In [ ]:
#get unique words
flatten = lambda l: [item for sublist in l for item in sublist]
vocabs = list(set(flatten(corpus_tokenized)))

print(len(vocabs))

In [ ]:
#numericalize the vocabs
word2index = {w: i for i, w in enumerate(vocabs)}

len(word2index)

In [ ]:
#append <UNK>
vocabs.append('<UNK>')
word2index['<UNK>'] = 9019

len(word2index)

### 2. GloVe

#### 2.1. Co-Occurence Matrix

In [ ]:
#count the frequency of each word
from collections import Counter

X_i = Counter(flatten(corpus_tokenized))

X_i['fellow']

In [ ]:
#generate skipgrams with a generic window size
def generate_skip_gram(window_size): 
    skip_grams = []
    for sentence in corpus_tokenized:
        #I include the first and last words
        #so that every word could be a center word
        for i in range(len(sentence)):
            center = sentence[i]
            context = []
            for j in range(window_size):
                if (i - window_size + j) >= 0:
                    context.append(sentence[i - window_size + j])
            for k in range(1, window_size + 1):
                if (i + k) < len(sentence):
                    context.append(sentence[i + k])
            for w in context:
                skip_grams.append((center, w))
        
    return skip_grams

In [ ]:
#prepare skipgrams with window size of 2
skip_grams = generate_skip_gram(2)

skip_grams[:5]

In [ ]:
#count co-occurences in the skipgrams
X_ik_skipgram = Counter(skip_grams)

len(X_ik_skipgram)

In [ ]:
print(X_ik_skipgram[('fellow', 'citizens')])
print(X_ik_skipgram[('fellow', 'communists')])

#### 2.2. Weighting Function

In [ ]:
#define the weighting function
def weighting(w_i, w_j, X_ik):
    
    x_ij = X_ik[(w_i, w_j)]

    #label smoothing if there is no co-occurence (i.e., x_ij is 0)
    if x_ij == 0:
        x_ij = 1
        
    #maximum co-occurrences is 100 according to the paper
    x_max = 100
    alpha = 0.75
    
    #if the co-occurrences does not exceed x_max, scale it down based on some alpha
    if x_ij < x_max:
        result = (x_ij/x_max) ** alpha
    else:
        result = 1 #the maximum probability
        
    return result

In [ ]:
#test the weighting function
w_i  = 'fellow'
w_j  = 'citizens'

print(weighting(w_i, w_j, X_ik_skipgram))

In [ ]:
w_i  = 'fellow'
w_j  = 'communists'

print(weighting(w_i, w_j, X_ik_skipgram))
print((1 / 100) ** 0.75)

In [ ]:
#apply this weighting to all possible pairs
from itertools import combinations_with_replacement

X_ik = {} #for keeping the co-occurrences
weighting_dic = {} #for keeping all the probabilities after passing through the weighting function

for bigram in combinations_with_replacement(vocabs, 2):  #we need to also think its reverse
    #if this bigram exists in X_ik_skipgrams
    #we gonna add this to our co-occurence matrix
    if X_ik_skipgram.get(bigram) is not None:
        cooc = X_ik_skipgram[bigram]  #get the co-occurrence
        X_ik[bigram] = cooc + 1 #this is again basically label smoothing
        X_ik[(bigram[1], bigram[0])] = cooc + 1  #trick to get all pairs
    else: #otherwise, put 0
        X_ik[bigram] = 0
        X_ik[(bigram[1], bigram[0])] = 0

    #apply the weighting function using this co-occurrence matrix thingy    
    weighting_dic[bigram] = weighting(bigram[0], bigram[1], X_ik)
    weighting_dic[(bigram[1], bigram[0])] = weighting(bigram[1], bigram[0], X_ik)

In [ ]:
#test the weighting function
print(X_ik_skipgram[('senate', 'house')])
print(X_ik_skipgram[('house', 'senate')])

print(X_ik[('senate', 'house')])
print(X_ik[('house', 'senate')])

print(weighting_dic[('senate', 'house')])
print(weighting_dic[('house', 'senate')])

print((5 / 100) ** 0.75)

In [ ]:
print(X_ik_skipgram[('communists', 'communists')])
print(X_ik[('communists', 'communists')])
print(weighting_dic[('communists', 'communists')])
print((1 / 100) ** 0.75)

In [ ]:
print(weighting_dic[('fellow', 'citizens')])

#### 2.3. Preparing Training Data

In [ ]:
#random batch for GloVe with generic batch size, corpus and skipgrams
import math

def random_batch_glove(batch_size, corpus, skip_grams, X_ik, weighting_dic):
    
    #change words in the skipgrams to idices
    skip_grams_id = [(word2index[skip_gram[0]], word2index[skip_gram[1]]) for skip_gram in skip_grams]
    
    #randomly pick "batch_size" indices
    number_of_choices = len(skip_grams_id)
    random_index = np.random.choice(number_of_choices, batch_size, replace=False) #no repeating indexes among these random indexes
    
    random_inputs = [] #xi, wi (in batches)
    random_labels = [] #xj, wj (in batches)
    random_coocs  = [] #Xij (in batches)
    random_weightings = [] #weighting_dic(Xij) (in batches)
    #for each of the sample in these indexes
    for i in random_index:
        random_inputs.append([skip_grams_id[i][0]])
        random_labels.append([skip_grams_id[i][1]])
        
        #get cooc
        #first check whether it exists...
        pair = skip_grams[i]
        try:
            cooc = X_ik[pair]
        except:
            cooc = 1 #label smoothing
            
        random_coocs.append([math.log(cooc)])
        #log according to the cost function equation
        #bracket because neural network requires size ( , 1)
        
        #get weighting
        weighting = weighting_dic[pair]
        random_weightings.append(weighting)

        
    return np.array(random_inputs), np.array(random_labels), np.array(random_coocs), np.array(random_weightings)
    

In [ ]:
#test the method
batch_size = 2
inputs, targets, coocs, weightings = random_batch_glove(batch_size, corpus_tokenized, skip_grams, X_ik, weighting_dic)

inputs, targets, coocs, weightings

#### 2.4. Model

In [ ]:
class GloVe(nn.Module):
    
    def __init__(self, vocab_size,embed_size):
        super(GloVe,self).__init__()
        self.embedding_v = nn.Embedding(vocab_size, embed_size) # center embedding
        self.embedding_u = nn.Embedding(vocab_size, embed_size) # context embedding
        
        self.v_bias = nn.Embedding(vocab_size, 1)
        self.u_bias = nn.Embedding(vocab_size, 1)
        
    def forward(self, center_words, context_words, coocs, weightings):
        center_embeds = self.embedding_v(center_words) # [batch_size, 1, emb_size]
        context_embeds = self.embedding_u(context_words) # [batch_size, 1, emb_size]
        
        center_bias = self.v_bias(center_words).squeeze(1)
        context_bias = self.u_bias(context_words).squeeze(1)
        
        inner_product = context_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2)
        #[batch_size, 1, emb_size] @ [batch_size, emb_size, 1] = [batch_size, 1, 1] = [batch_size, 1]
        
        #note that coocs is already log
        loss = weightings * torch.pow(inner_product + center_bias + context_bias - coocs, 2)
        
        return torch.sum(loss)

#### 2.5. Training

In [ ]:
#set training parameters
batch_size  = 10
vocab_size  = len(vocabs)
emb_size    = 2
model       = GloVe(vocab_size, emb_size)
optimizer   = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
#calculate epoch time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time // 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
#train the model
import time

num_epochs = 500
start = time.time()
for epoch in range(num_epochs):
    
    inputs, targets, coocs, weightings = random_batch_glove(batch_size, corpus_tokenized, skip_grams, X_ik, weighting_dic)
    input_batch     = torch.LongTensor(inputs)
    target_batch    = torch.LongTensor(targets)
    cooc_batch      = torch.FloatTensor(coocs)
    weighting_batch = torch.FloatTensor(weightings)

    optimizer.zero_grad()
    loss = model(input_batch, target_batch, cooc_batch, weighting_batch)
    
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        end = time.time()
        epoch_mins, epoch_secs = epoch_time(start, end)

        print(f"Epoch: {epoch + 1} | cost: {loss:.6f} | time: {epoch_mins}m {epoch_secs}s")

#### 2.6. Plotting the Embeddings

#### 2.7. Cosine Similarity